In [40]:
#Import the basics
import numpy as np
import matplotlib as plt
import pandas as pd

#Get sklearn
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut

#Get tensorflow and keras
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import optimizers
# Helper libraries

print(tf.__version__)

2.6.0


In [41]:
#Read in data from the csv file
data = pd.read_csv("Structured_Data.csv")

In [42]:
#Check things look good
data.head(3)

,Target-20_Open_VET,Target-20_High_VET,Target-20_Low_VET,Target-20_Close_VET,Target-20_Volume_VET,Target-20_Open_BTC,Target-20_High_BTC,Target-20_Low_BTC,Target-20_Close_BTC,Target-20_Volume_BTC,...,Target-1_High_Difference_From_Lower_Bollinger_Band,Target-1_Low_Difference_From_Lower_Bollinger_Band,Target-1_Close_Difference_From_Lower_Bollinger_Band,Target-1_Open_Difference_From_SMA20,Target-1_High_Difference_From_SMA20,Target-1_Low_Difference_From_SMA20,Target-1_Close_Difference_From_SMA20,Target-20_Original_Open_VET,Regression_Target,Classification_Target
0,1.0,1.000000,0.948314,0.985892,19980000,6551.520020,6719.959961,6498.640137,6719.959961,4097820000,...,-0.083217,0.069804,-0.072636,1.090632,0.154802,0.307824,0.165384,0.014743,0.953673,1.0
1,1.0,1.081295,0.978835,1.059167,24453700,6719.950195,6789.629883,6700.959961,6763.189941,3312600000,...,-0.114661,-0.040307,-0.094252,1.102816,0.130051,0.204405,0.150460,0.014552,0.960555,0.0
2,1.0,1.003639,0.943015,0.989799,20503900,6754.640137,6774.750000,6620.750000,6707.259766,3295500000,...,-0.120802,-0.067261,-0.096111,1.037097,0.118242,0.171784,0.142934,0.015390,0.927550,1.0


In [43]:
#Create target column for regression
y = data["Classification_Target"]
y

0       1.0
1       0.0
2       1.0
3       0.0
4       0.0
       ... 
1139    1.0
1140    0.0
1141    0.0
1142    1.0
1143    0.0
Name: Classification_Target, Length: 1144, dtype: float64

In [44]:
#Drop the target columns to create input data
X = data.drop(columns=["Regression_Target", "Classification_Target"])
X.head(3)

,Target-20_Open_VET,Target-20_High_VET,Target-20_Low_VET,Target-20_Close_VET,Target-20_Volume_VET,Target-20_Open_BTC,Target-20_High_BTC,Target-20_Low_BTC,Target-20_Close_BTC,Target-20_Volume_BTC,...,Target-1_Close_Difference_From_Upper_Bollinger_Band,Target-1_Open_Difference_From_Lower_Bollinger_Band,Target-1_High_Difference_From_Lower_Bollinger_Band,Target-1_Low_Difference_From_Lower_Bollinger_Band,Target-1_Close_Difference_From_Lower_Bollinger_Band,Target-1_Open_Difference_From_SMA20,Target-1_High_Difference_From_SMA20,Target-1_Low_Difference_From_SMA20,Target-1_Close_Difference_From_SMA20,Target-20_Original_Open_VET
0,1.0,1.000000,0.948314,0.985892,19980000,6551.520020,6719.959961,6498.640137,6719.959961,4097820000,...,0.403403,0.852612,-0.083217,0.069804,-0.072636,1.090632,0.154802,0.307824,0.165384,0.014743
1,1.0,1.081295,0.978835,1.059167,24453700,6719.950195,6789.629883,6700.959961,6763.189941,3312600000,...,0.395172,0.858104,-0.114661,-0.040307,-0.094252,1.102816,0.130051,0.204405,0.150460,0.014552
2,1.0,1.003639,0.943015,0.989799,20503900,6754.640137,6774.750000,6620.750000,6707.259766,3295500000,...,0.381979,0.798052,-0.120802,-0.067261,-0.096111,1.037097,0.118242,0.171784,0.142934,0.015390


In [45]:
#Split the data into training (80%) and testing sets (20%), includes shuffling
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, train_size=0.8)

In [46]:
#Split the training data into training (80%) and validation (20%)
xtraintrain, xval, ytraintrain, yval = train_test_split(xtrain, ytrain, test_size=0.2)

In [47]:
def mapto1hot(ydata):

    ydata_1hot = tf.keras.utils.to_categorical(ydata, num_classes=2)

    return ydata_1hot

In [48]:
ytrain_1hot = mapto1hot(ytrain)
ytest_1hot = mapto1hot(ytest)

In [51]:
layers = [
    Flatten()
    Dense(1000, activation='relu', kernel_regularizer='l1'),
    Dense(1000, activation='relu'),
    Dense(1000, activation='relu'),
    Dense(2)
]

model = Sequential(layers)

opt = tf.keras.optimizers.SGD(learning_rate=0.01)
#refer to https://www.tensorflow.org/tutorials/keras/classification tutorial to check on how to use compile function
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=['accuracy'])

# This will start the training and save each epoch output in the history list
history = model.fit(xtrain, ytrain_1hot, batch_size=10, epochs=20, validation_data=(xtest, ytest_1hot))

Epoch 1/20
92/92 [==============================] - 2s 15ms/step - loss: nan - accuracy: 0.4765 - val_loss: nan - val_accuracy: 0.5328
Epoch 2/20
92/92 [==============================] - 1s 13ms/step - loss: nan - accuracy: 0.4765 - val_loss: nan - val_accuracy: 0.5328
Epoch 3/20
92/92 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.4765 - val_loss: nan - val_accuracy: 0.5328
Epoch 4/20
92/92 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.4765 - val_loss: nan - val_accuracy: 0.5328
Epoch 5/20
92/92 [==============================] - 1s 13ms/step - loss: nan - accuracy: 0.4765 - val_loss: nan - val_accuracy: 0.5328
Epoch 6/20
92/92 [==============================] - 1s 13ms/step - loss: nan - accuracy: 0.4765 - val_loss: nan - val_accuracy: 0.5328
Epoch 7/20
92/92 [==============================] - 1s 13ms/step - loss: nan - accuracy: 0.4765 - val_loss: nan - val_accuracy: 0.5328
Epoch 8/20
92/92 [==============================] - 1s 